In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Assignment-Bank").getOrCreate()

In [0]:
df = spark.read.csv('dbfs:/FileStore/shared_uploads/mohbilal@deloitte.com/bank_customer.csv',header=True,inferSchema=True)
df_fd = spark.read.csv('dbfs:/FileStore/shared_uploads/mohbilal@deloitte.com/bank_fd_account.csv',header=True,inferSchema=True)
df_rd = spark.read.csv('dbfs:/FileStore/shared_uploads/mohbilal@deloitte.com/bank_rd_account.csv',header=True,inferSchema=True)
df_sb = spark.read.csv('dbfs:/FileStore/shared_uploads/mohbilal@deloitte.com/bank_sb_account.csv',header=True,inferSchema=True)

In [0]:
# fixed deposit(fd)
# recurring deposit(rd)
# Saving bank(sb)

In [0]:
df.show()

+-------+-------------------+--------+----------+--------+---------+---------+
|CUST_ID|         CUST_FNAME|INITIALS|CUST_LNAME|CUST_SEX| CUST_DOB|CUST_TYPE|
+-------+-------------------+--------+----------+--------+---------+---------+
|   1001|               John|    null|     Smith|       M|21-OCT-78|      IND|
|   1002|           Franklin|       J|      Dang|       M|21-OCT-78|      IND|
|   1003|              Alice|       M|    Powell|       F|09-APR-76|      IND|
|   1004|             Ramesh|       R|   Narayan|       M|08-DEC-71|      IND|
|   1005|              Nancy|       S|    Taylor|       F|17-AUG-79|      IND|
|   1006|              Ahmad|     M A|    Jabbar|       M|25-JUN-72|      IND|
|   1007|       DealWell Co.|    null|      null|    null|     null|     CORP|
|   1008|               Aman|       K|    Mathur|       M|30-OCT-80|      IND|
|   1009|             Smitha|    null|    Ramesh|       F|03-MAR-76|      IND|
|   1010|Popular Enterprises|    null|      null|   

In [0]:
df.createOrReplaceTempView("customer")
df_fd.createOrReplaceTempView("fixed_acc")
df_sb.createOrReplaceTempView("savings_acc")
df_rd.createOrReplaceTempView("recurring_acc")

# A)First Question
- Get a list of customers who have active FD account with more than 25000 as the initial amount.
- The list should have the customer id, first name, FD account number and initial amount;
- The list should be sorted on the initial amount with highest amount first.

In [0]:
%sql
select * from customer;

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
%sql
select * from fixed_acc;

ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS
5212340001,1002,22-JUN-00,30,15000,Closed
5212340002,1002,11-MAY-02,66,25000,Active
5212340003,1007,26-NOV-03,36,80555,Closed
5212340004,1003,29-NOV-04,60,30000,Active
5212340005,1009,23-JUN-04,120,45000,Active
5212340006,1001,26-SEP-05,24,45000,Active
5212340007,1010,18-OCT-05,84,15000000,Active
5212340008,1008,18-DEC-06,36,98765,Active
5212340009,1006,14-FEB-07,120,25000,Active
5212340010,1007,21-SEP-08,60,2500000,Active


In [0]:
%sql
select cust_id,cust_fname,account_no,initial_amt,acc_status from customer natural join fixed_acc 
where (initial_amt>25000 and acc_status='Active') 
order by initial_amt desc;

cust_id,cust_fname,account_no,initial_amt,acc_status
1010,Popular Enterprises,5212340007,15000000,Active
1007,DealWell Co.,5212340010,2500000,Active
1008,Aman,5212340008,98765,Active
1009,Smitha,5212340005,45000,Active
1001,John,5212340006,45000,Active
1003,Alice,5212340004,30000,Active


In [0]:
#Doing using pyspark
df.join(df_fd, df.CUST_ID == df_fd.CUST_ID,
               "inner").filter((df_fd.INITIAL_AMT>25000) & (df_fd.ACC_STATUS=='Active')).orderBy(df_fd.INITIAL_AMT.desc()).show()

# Inner join
# This will join the two PySpark dataframes on key columns, which are common in both dataframes.
# Syntax: dataframe1.join(dataframe2,dataframe1.column_name ==  dataframe2.column_name,”inner”)

+-------+-------------------+--------+----------+--------+---------+---------+----------+-------+---------+------------+-----------+----------+
|CUST_ID|         CUST_FNAME|INITIALS|CUST_LNAME|CUST_SEX| CUST_DOB|CUST_TYPE|ACCOUNT_NO|CUST_ID| START_DT|FD_TERM_MNTH|INITIAL_AMT|ACC_STATUS|
+-------+-------------------+--------+----------+--------+---------+---------+----------+-------+---------+------------+-----------+----------+
|   1010|Popular Enterprises|    null|      null|    null|     null|     CORP|5212340007|   1010|18-OCT-05|          84|   15000000|    Active|
|   1007|       DealWell Co.|    null|      null|    null|     null|     CORP|5212340010|   1007|21-SEP-08|          60|    2500000|    Active|
|   1008|               Aman|       K|    Mathur|       M|30-OCT-80|      IND|5212340008|   1008|18-DEC-06|          36|      98765|    Active|
|   1009|             Smitha|    null|    Ramesh|       F|03-MAR-76|      IND|5212340005|   1009|23-JUN-04|         120|      45000|    

# B)Second Question
- Get a list of customers who have active accounts in all three types FD, RD and DB, along with their net worth.
- Net worth is calculated as the total of FD Initial Amount, RD Current Balance and SB Current Balance.
- The list should have the customer id, first name, FD account number, FD initial amount, RD Current Balance, 
  SB Current Balance and Networth for all active accounts of the customers; the list should be sorted on net worth with highest amount
  appearing first.

In [0]:
%sql
SELECT c.CUST_ID, c.CUST_FNAME,f.ACCOUNT_NO,f.INITIAL_AMT as FD_INITIAL_AMT,
r.CURR_BALANCE as RD_CURRENT_BALANCE,s.CURR_BAL_AMT as SB_CURRENT_BALANCE,
f.INITIAL_AMT+r.CURR_BALANCE+s.CURR_BAL_AMT as TOTAL FROM customer c
JOIN fixed_acc f ON c.CUST_ID = f.CUST_ID 
JOIN recurring_acc r ON f.CUST_ID = r.RD_CUST_ID
JOIN savings_acc s ON r.RD_CUST_ID = s.PRIMARY_CUST_ID
WHERE f.ACC_STATUS = 'Active' and r.ACC_STATUS = 'Active' and s.ACC_STATUS = 'Active';

CUST_ID,CUST_FNAME,ACCOUNT_NO,FD_INITIAL_AMT,RD_CURRENT_BALANCE,SB_CURRENT_BALANCE,TOTAL
1008,Aman,5212340008,98765,148000,84924.0,331689.0
1006,Ahmad,5212340009,25000,75000,73535.0,173535.0


# C)Third Question
- Create a new DataFrame that has a column “CUST_TYP_INDEX” which is a unique number
  assigned to each customer type. For example: 1.0 for Corporate type of customer (CORP),
  2 for individual customer (IND).

In [0]:
from pyspark.sql.functions import when

customer_indexed = df.withColumn(
    "CUST_TYP_INDEX",
    when(df["CUST_TYPE"] == "CORP", 1.0).otherwise(2.0)
).show()


+-------+-------------------+--------+----------+--------+---------+---------+--------------+
|CUST_ID|         CUST_FNAME|INITIALS|CUST_LNAME|CUST_SEX| CUST_DOB|CUST_TYPE|CUST_TYP_INDEX|
+-------+-------------------+--------+----------+--------+---------+---------+--------------+
|   1001|               John|    null|     Smith|       M|21-OCT-78|      IND|           2.0|
|   1002|           Franklin|       J|      Dang|       M|21-OCT-78|      IND|           2.0|
|   1003|              Alice|       M|    Powell|       F|09-APR-76|      IND|           2.0|
|   1004|             Ramesh|       R|   Narayan|       M|08-DEC-71|      IND|           2.0|
|   1005|              Nancy|       S|    Taylor|       F|17-AUG-79|      IND|           2.0|
|   1006|              Ahmad|     M A|    Jabbar|       M|25-JUN-72|      IND|           2.0|
|   1007|       DealWell Co.|    null|      null|    null|     null|     CORP|           1.0|
|   1008|               Aman|       K|    Mathur|       M|30

# E)Question 4
- Get a list of customer ids along with the account number and amount of their active
  accounts of each type of account they have.
  For example, if a customer has all 3 types of accounts active, then the list should contain
  customer id, active FD account number, initial amount, status, active RD account number,
  balance amount, status, active SB account number, balance amount, status.
  If a customer has only 2 types of account
  active then the list should have details of only
  these 2 accounts. The list should be sorted by the customer id and for each customer id it
  should be sorted on account number.

In [0]:
%sql
SELECT fixed_acc.CUST_ID, fixed_acc.ACCOUNT_NO, fixed_acc.INITIAL_AMT AS AMT, fixed_acc.ACC_STATUS FROM fixed_acc 
WHERE fixed_acc.ACC_STATUS = 'Active'
UNION
SELECT recurring_acc.RD_CUST_ID, recurring_acc.ACCOUNT_NO, recurring_acc.CURR_BALANCE, recurring_acc.ACC_STATUS FROM recurring_acc
WHERE recurring_acc.ACC_STATUS = 'Active'
UNION
SELECT savings_acc.PRIMARY_CUST_ID, savings_acc.ACCOUNT_NO, savings_acc.CURR_BAL_AMT, savings_acc.ACC_STATUS FROM savings_acc
WHERE savings_acc.ACC_STATUS = 'Active'
ORDER BY CUST_ID, ACCOUNT_NO

CUST_ID,ACCOUNT_NO,AMT,ACC_STATUS
1001,1000012001,25000.0,Active
1001,5212340006,45000.0,Active
1002,1000012002,31109.43,Active
1002,5212340002,25000.0,Active
1003,5212340004,30000.0,Active
1003,6000010002,208000.0,Active
1004,1000012010,54674.0,Active
1005,1000012008,10235.0,Active
1006,1000012009,73535.0,Active
1006,5212340009,25000.0,Active


# F)Question 5
Modify the above to add the customer first name and also to add a column named acc_type
which should show
- 'FD' if it is an FD account
- 'RD' if it is an RD account
- 'SB' if it is an SB account

In [0]:
%sql
SELECT fixed_acc.CUST_ID, fixed_acc.ACCOUNT_NO, fixed_acc.INITIAL_AMT AS AMT, fixed_acc.ACC_STATUS, 'FD' as ACC_TYPE  FROM fixed_acc 
WHERE fixed_acc.ACC_STATUS = 'Active'
UNION
SELECT recurring_acc.RD_CUST_ID, recurring_acc.ACCOUNT_NO, recurring_acc.CURR_BALANCE, recurring_acc.ACC_STATUS,'RD' FROM recurring_acc
WHERE recurring_acc.ACC_STATUS = 'Active'
UNION
SELECT savings_acc.PRIMARY_CUST_ID, savings_acc.ACCOUNT_NO, savings_acc.CURR_BAL_AMT, savings_acc.ACC_STATUS,'SB' FROM savings_acc
WHERE savings_acc.ACC_STATUS = 'Active'
ORDER BY CUST_ID, ACCOUNT_NO

CUST_ID,ACCOUNT_NO,AMT,ACC_STATUS,ACC_TYPE
1001,1000012001,25000.0,Active,SB
1001,5212340006,45000.0,Active,FD
1002,1000012002,31109.43,Active,SB
1002,5212340002,25000.0,Active,FD
1003,5212340004,30000.0,Active,FD
1003,6000010002,208000.0,Active,RD
1004,1000012010,54674.0,Active,SB
1005,1000012008,10235.0,Active,SB
1006,1000012009,73535.0,Active,SB
1006,5212340009,25000.0,Active,FD
